In [19]:
%pip install biodivine_aeon==1.0.0a8

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: biodivine_aeon==1.0.0a8 in c:\users\x240\.virtualenvs\casq-fork\lib\site-packages (1.0.0a8)




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from biodivine_aeon import *
from pathlib import Path

In [21]:
model = BooleanNetwork.from_file("model_map1.aeon")
print(model)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=37, implicit_parameters=22)


In [22]:
stg = AsynchronousGraph(model)

First, we identify the unknown functions that can be safely eliminated because they have only one interpretations that satisfies all the requirements regarding their input monotonicity and essentiality.

In [23]:
to_eliminate = []
for param in model.explicit_parameters():
    print(model.get_explicit_parameter_name(param))
    count = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    if count == 1:
        to_eliminate.append(param)

cat_Gab2_phosphorylated_51
 >  ColorModel({'cat_Gab2_phosphorylated_51': 'x_0'})
cat_FceRI_Syk_complex_Cytosol_1_24
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'x_0'})
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'true'})
cat_DAG_simple_molecule_65
 >  ColorModel({'cat_DAG_simple_molecule_65': '(x_0 & x_1)'})
 >  ColorModel({'cat_DAG_simple_molecule_65': '((!x_0 & x_1) | x_0)'})
cat_IKK_Beta_phosphorylated_39
 >  ColorModel({'cat_IKK_Beta_phosphorylated_39': 'x_0'})
cat_LAT2_phosphorylated_palmytoylated_47
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '(x_0 & x_1)'})
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '((!x_0 & x_1) | x_0)'})
cat_Fyn_phosphorylated_phosphorylated_palmytoylated_62
 >  ColorModel({'cat_Fyn_phosphorylated_phosphorylated_palmytoylated_62': 'x_0'})
cat_MKK7_MAP2K7__phosphorylated_67
 >  ColorModel({'cat_MKK7_MAP2K7__phosphorylated_67': 'x_0'})
cat_MEK1_MAP2K1__phosphorylated_50
 >  ColorModel({'cat_MEK1_MAP2K1__phosp

In [24]:
print(to_eliminate)

[ParameterId(16), ParameterId(17), ParameterId(15), ParameterId(27), ParameterId(23), ParameterId(28), ParameterId(12), ParameterId(3), ParameterId(25), ParameterId(10), ParameterId(32), ParameterId(33), ParameterId(30), ParameterId(26), ParameterId(14), ParameterId(21), ParameterId(31), ParameterId(0), ParameterId(19), ParameterId(6), ParameterId(13), ParameterId(1), ParameterId(2), ParameterId(24), ParameterId(36), ParameterId(4)]


In [25]:
m = next(stg.mk_unit_colors().items(retained = to_eliminate))

In [26]:
model_simplified = m.instantiate(model)

In [27]:
Path("model_map1.simplified.aeon").write_text(model_simplified.to_aeon())

13664

In [28]:
stg = AsynchronousGraph(model_simplified)

In [29]:
# This identifies model outputs. In the next step, we restrict them manually to
# a subset of outputs that are biologically interpretable.
outputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.successors(var)) == 0:
        outputs.append(var)
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Fyn_phosphorylated_phosphorylated_palmytoylated',
 'I_kB_phosphorylated',
 'Jun_Fos_complex',
 'LAT2_Grb2_Shc_SOS_H_RAS_RAF1_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex',
 'c_JUN_phosphorylated_nucleus']

In [30]:
# Actually relevant outputs:
outputs = ['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']

In [31]:
inputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.predecessors(var)) == 0:
        inputs.append(var)
inputs

['Akt',
 'Bcr',
 'Btk',
 'CRAC',
 'Csk',
 'Dok1_RasGAP_complex',
 'Fyn_phosphorylated_palmytoylated',
 'GRAP2',
 'H_RAS_Plasma_space_Membrane',
 'LAT2_Grb2_complex',
 'PAK2',
 'PIP2_simple_molecule',
 'PI_3K',
 'PKC_space_Theta',
 'PLCG1',
 'RAF1_Cytosol',
 'RASGRP1',
 'Rac1',
 'Shc_Grb2_complex',
 'Syk',
 'Vav1',
 'cCbl']

In [32]:
params = []
for p in model_simplified.explicit_parameter_names():
    params.append((p, model_simplified.get_explicit_parameter_arity(p)))
params

[('cat_Calmodulin_Calcineurin_complex_19', 2),
 ('cat_DAG_simple_molecule_65', 2),
 ('cat_Elk1_phosphorylated_53', 3),
 ('cat_Erk_MAPK1_3__phosphorylated_49', 2),
 ('cat_FceRI_Syk_complex_Cytosol_1_24', 1),
 ('cat_IP3_simple_molecule_57', 2),
 ('cat_JNK_phosphorylated_70', 2),
 ('cat_LAT2_phosphorylated_palmytoylated_47', 2),
 ('cat_PIP3_Akt_complex_8', 1),
 ('cat_SOS_Grb2_complex_5', 1),
 ('cat_Shc_phosphorylated_56', 2)]

In [33]:
ctx = SymbolicSpaceContext(model_simplified)
stg = AsynchronousGraph(model_simplified, ctx)

Compute minimal trap spaces across all interpretations. These should almost exactly correspond to model attractors (this can take a few minutes).

In [34]:
traps = TrapSpaces.essential_symbolic(ctx, stg, ctx.mk_unit_colored_spaces(stg))

In [35]:
min_traps = TrapSpaces.minimize(ctx, traps)

In [36]:
min_traps

ColoredSpaceSet(cardinality=228493099008, colors=77309411328, spaces=35929632, symbolic_size=588576)

Compute the number of output value combinations that can actually appear in the model, and the distribution of output values within these combinations.

In [37]:
phenotype_iterator = min_traps.spaces().items(retained = outputs)
phenotype_count = sum(1 for _ in phenotype_iterator)
print(f"Phenotypes: {phenotype_count}")

Phenotypes: 84


In [38]:
is_zero = { var: 0 for var in outputs }
is_one = { var: 0 for var in outputs }
is_any = { var: 0 for var in outputs }
for phenotype in min_traps.spaces().items(retained = outputs):
    phenotype_traps = min_traps.intersect_spaces(phenotype.to_symbolic())
    for (var, value) in phenotype.items():        
        var = ctx.get_network_variable_name(var)
        if value == True:
            is_one[var] += 1
        elif value == False:
            is_zero[var] += 1
        else:
            is_any[var] += 1

print("Zero:", is_zero)
print("One:", is_one)
print("Any:", is_any)

Zero: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 52, 'Elk1_phosphorylated': 20, 'Jun_Fos_complex': 52, 'NFAT_nucleus': 26, 'NF_kappa_B_nucleus': 36, 'PIP3_Akt_complex': 42}
One: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 20, 'Elk1_phosphorylated': 24, 'Jun_Fos_complex': 12, 'NFAT_nucleus': 58, 'NF_kappa_B_nucleus': 30, 'PIP3_Akt_complex': 42}
Any: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 12, 'Elk1_phosphorylated': 40, 'Jun_Fos_complex': 20, 'NFAT_nucleus': 0, 'NF_kappa_B_nucleus': 18, 'PIP3_Akt_complex': 0}


In [39]:
def cardinality_without_inputs(ctx, inputs, set):
    # Project the BDD to remove inputs.
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    all_vars = ctx.bdd_variable_set().variable_count()
    function_vars = sum(len(x) for x in ctx.explicit_functions_bdd_variables().values())

    general_cardinality = set_bdd.cardinality()
    return int(general_cardinality / (2 ** (all_vars - function_vars)))

def prune_inputs(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    return ColorSet(ctx, set_bdd)

def prune_inputs_forall(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_for_all(ignore_vars)

    return ColorSet(ctx, set_bdd)

Build a table showing the multiplicity of various attractor combinations for each output variable:

In [40]:
print(f"0\t1\t*\t0-1\t1-*\t0-*\t0-1-*\tname")

for var in outputs:
    p_var = ctx.get_positive_space_variable(var)
    n_var = ctx.get_negative_space_variable(var)
    var_is_one = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: False }))
    var_is_zero = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: False, n_var: True }))
    var_is_any = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: True }))

    one_colors = min_traps.intersect_spaces(var_is_one).colors()
    zero_colors = min_traps.intersect_spaces(var_is_zero).colors()
    any_colors = min_traps.intersect_spaces(var_is_any).colors()

    # one_colors = prune_inputs(ctx, inputs, one_colors)
    # zero_colors = prune_inputs(ctx, inputs, zero_colors)
    # any_colors = prune_inputs(ctx, inputs, any_colors)

    one = one_colors.minus(zero_colors).minus(any_colors)
    zero = zero_colors.minus(one_colors).minus(any_colors)
    any = any_colors.minus(zero_colors).minus(one_colors)
    zero_one = one_colors.intersect(zero_colors).minus(any_colors)
    one_any = one_colors.intersect(any_colors).minus(zero_colors)
    zero_any = zero_colors.intersect(any_colors).minus(one_colors)
    zero_one_any = zero_colors.intersect(one_colors).intersect(any_colors)
    
    # c_one = cardinality_without_inputs(ctx, inputs, one)
    # c_zero = cardinality_without_inputs(ctx, inputs, zero)
    # c_any = cardinality_without_inputs(ctx, inputs, any)
    # c_zero_one = cardinality_without_inputs(ctx, inputs, zero_one)
    # c_one_any = cardinality_without_inputs(ctx, inputs, one_any)
    # c_zero_any = cardinality_without_inputs(ctx, inputs, zero_any)
    # c_zero_one_any = cardinality_without_inputs(ctx, inputs, zero_one_any)

    c_one = one.cardinality()
    c_zero = zero.cardinality()
    c_any = any.cardinality()
    c_zero_one = zero_one.cardinality()
    c_one_any = one_any.cardinality()
    c_zero_any = zero_any.cardinality()
    c_zero_one_any = zero_one_any.cardinality()

    print(f"{c_zero}\t{c_one}\t{c_any}\t{c_zero_one}\t{c_one_any}\t{c_zero_any}\t{c_zero_one_any}\t{var}")    

0	1	*	0-1	1-*	0-*	0-1-*	name
76880019456	207618048	7077888	207618048	0	7077888	0	Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex
76910952448	351272960	47185920	0	0	0	0	Elk1_phosphorylated
77224476672	56623104	28311552	0	0	0	0	Jun_Fos_complex
0	38654705664	0	38654705664	0	0	0	NFAT_nucleus
75591843840	1703411712	14155776	0	0	0	0	NF_kappa_B_nucleus
75723964416	1585446912	0	0	0	0	0	PIP3_Akt_complex


Finally, let us compare the partial network to the original CaSQ network:

In [41]:
model_known = BooleanNetwork.from_file("model_map1.full.aeon")
print(model_known)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=0, implicit_parameters=22)


In [42]:
ctx_known = SymbolicSpaceContext(model_known)
stg_known = AsynchronousGraph(model_known, ctx_known)

In [43]:
traps_known = TrapSpaces.essential_symbolic(ctx_known, stg_known, ctx_known.mk_unit_colored_spaces(stg_known))

In [44]:
min_traps_known = TrapSpaces.minimize(ctx_known, traps_known)

In [45]:
min_traps_transferred = ColoredSpaceSet(ctx, ctx.transfer_from(min_traps_known.to_bdd(), ctx_known))

In [46]:
equivalent = prune_inputs_forall(ctx, inputs, min_traps.intersect(min_traps_transferred).colors())
print("All models:", cardinality_without_inputs(ctx, inputs, stg.mk_unit_colors()))
print("Equivalent models:", cardinality_without_inputs(ctx, inputs, equivalent))

All models: 18432
Equivalent models: 128


Let's examine which funtions are fixed/free in the 128 equivalent models:

In [47]:
for param in model_simplified.explicit_parameters():
    print(model_simplified.get_explicit_parameter_name(param))
    count = 0
    for m in equivalent.items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    total = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        total += 1
    print(f"{count}/{total}")

cat_Elk1_phosphorylated_53
 >  ColorModel({'cat_Elk1_phosphorylated_53': '((!x_0 & (x_1 & x_2)) | x_0)'})
 >  ColorModel({'cat_Elk1_phosphorylated_53': '((!x_0 & (!x_1 & x_2)) | ((!x_0 & x_1) | x_0))'})
2/9
cat_Shc_phosphorylated_56
 >  ColorModel({'cat_Shc_phosphorylated_56': '((!x_0 & x_1) | x_0)'})
1/2
cat_IP3_simple_molecule_57
 >  ColorModel({'cat_IP3_simple_molecule_57': '(x_0 & x_1)'})
 >  ColorModel({'cat_IP3_simple_molecule_57': '((!x_0 & x_1) | x_0)'})
2/2
cat_Erk_MAPK1_3__phosphorylated_49
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '(x_0 & x_1)'})
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '((!x_0 & x_1) | x_0)'})
2/2
cat_FceRI_Syk_complex_Cytosol_1_24
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'x_0'})
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'true'})
2/2
cat_LAT2_phosphorylated_palmytoylated_47
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '((!x_0 & x_1) | x_0)'})
1/2
cat_DAG_simple_molecule_65
 >  ColorModel({'cat